### L0: Dummy Clasification example

In [ ]:
import torch 
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
sentences = ['I think I am in love', 'I think about dropping out of school to pursue my dream']

batch = tokenizer(sentences, padding='max_length', max_length=512, return_tensors='pt')
batch['labels'] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()


In [ ]:
print(model.config)

### L1: Classification - Are the 2 sentences conveying similar meaning?

In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset("glue", "mrpc")

In [ ]:
print(raw_datasets["train"].features)

In [ ]:
# --- explicit
sentence1, sentence2 = raw_datasets["train"][0]["sentence1"], raw_datasets["train"][0]["sentence2"]
ex_inputs = tokenizer(sentence1, sentence2, padding=True, truncation=True)
tokens = tokenizer.convert_ids_to_tokens(ex_inputs["input_ids"])

In [ ]:
# --- tokenizer all dataset using mapping

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding='max_length')
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

### L2: Testing out Dynamic Padding with data collator

In [ ]:
from transformers import DataCollatorWithPadding

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
samples = tokenized_datasets["train"][:10]

# create dictionary with input_ids, token_type_ids, attention_mask, label
samples = {k:v for k, v in samples.items() if k not in ['idx', 'sentence1', 'sentence2']}


In [ ]:
print(samples.keys())
print([len(sample) for sample in samples['input_ids']])

In [ ]:
batched = data_collator(samples)

### L3: Fine-tuning using TrainerAPI

Docs:
- [Training Arguments Docs](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments)
- [Fine-Tuning LLM Cookbook](https://huggingface.co/learn/cookbook/en/fine_tuning_code_llm_on_single_gpu)
- [Trainer Docs](https://huggingface.co/docs/transformers/en/main_classes/trainer)

In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model, 
    training_args, 
    train_dataset=tokenized_datasets["train"], 
    eval_dataset=tokenized_datasets["validation"], 
    data_collator=data_collator, 
    processing_class=tokenizer, 
)

trainer.train()

In [ ]:
import numpy as np
import evaluate

predictions = trainer.predict(tokenized_datasets["validation"])


In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

### L3.5: Cleaner TrainerAPI

In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
    
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    metric = evaluate.load("glue", "mrpc")
    preds = np.argmax(logits, axis=-1)
    return metrics.compute(predictions=preds, references=labels)

In [ ]:
model = model.to('cpu') # fix fp16 error => ValueError: fp16 mixed precision with MPS device requires a Pytorch >= 2.5.0

In [ ]:
training_args = TrainingArguments(
    "test-trainer", 
    eval_strategy="epoch",  
    num_train_epochs=1, 
    learning_rate=5e-5,
    # optim='adamw_torch', 
    fp16=False, 
    # per_device_train_batch_size=4,
    # gradient_accumulation_steps=4,
    # lr_scheduler_type="cosine"
)
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

### L4: Fine-Tuning using modern PyTorch (TODO)